# Monitoring margin cushion event breach and cancels all orders

In [1]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = "NSE"

# Set the root
from from_root import from_root
ROOT = from_root()

import pandas as pd
from loguru import logger

pd.options.display.max_columns = None
pd.set_option('display.precision', 2)

from pathlib import Path
import sys

# Add `src` and ROOT to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path

if "src" not in Path.cwd().parts:
    src_path = str(Path(get_path("purelib")) / "_src.pth")
    with open(src_path, "w") as f:
        f.write(str(ROOT / "src\n"))
        f.write(str(ROOT))
        if str(ROOT) not in sys.path:
            sys.path.insert(1, str(ROOT))

# Start the Jupyter loop
from ib_async import util

util.startLoop()

logger.add(sink=ROOT / "log" / "ztest.log", mode="w")

1

In [2]:
import asyncio
from ib_async import IB, util

# Configuration
MARGIN_CUSHION_THRESHOLD = 0.1  # Set your threshold here (10% for example)
CHECK_INTERVAL = 60  # Check every 60 seconds

async def monitor_margin_cushion(ib):
    while True:
        # Fetch account details
        account_summary = ib.accountSummary()
        margin_cushion = None
        
        # Extract Margin Cushion from account summary
        for item in account_summary:
            if item.tag == "Cushion":
                margin_cushion = float(item.value)
                break
        
        print(f"Current Margin Cushion: {margin_cushion}")

        # Check if Margin Cushion is below the threshold
        if margin_cushion is not None and margin_cushion < MARGIN_CUSHION_THRESHOLD:
            print("Margin Cushion breach detected! Cancelling all orders...")
            await cancel_all_orders(ib)

        await asyncio.sleep(CHECK_INTERVAL)

async def cancel_all_orders(ib):
    # Cancel all open orders
    open_orders = ib.openOrders()
    for order in open_orders:
        print(f"Cancelling order: {order}")
        ib.cancelOrder(order)

async def main():
    # Connect to IBKR
    ib = IB()
    ib.connect('127.0.0.1', 1300, clientId=10)  # Adjust host and port as needed

    try:
        await monitor_margin_cushion(ib)
    finally:
        ib.disconnect()

if __name__ == "__main__":
    asyncio.run(main())


Current Margin Cushion: 0.759853
